In [ ]:
#from google.colab import drive
#drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import pandas as pd
import os
import numpy as np
import sqlite3

In [ ]:
import geopy

In [ ]:
pd_data = pd.read_csv("../data/311_Service_Requests_small_sample.csv")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (9,18,32,33,35,36,37,38) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
pd_data = pd_data.rename(columns={"Created Date":"Created_Date",
                                  "Closed Date":"Closed_Date",
                                  "Agency Name":"Agency_Name",
                                  "Complaint Type":"Complaint_Type",
                                  "Location Type":"Location_Type",
                                  "Incident Zip":"Incident_Zip",
                                  "Incident Address":"Incident_Address",
                                  "Street Name":"Street_Name",
                                  "Cross Street 1":"Cross_Street_1",
                                  "Cross Street 2":"Cross_Street_2",
                                  "Intersection Street 1":"Intersection_Street_1",
                                  "Intersection Street 2":"Intersection_Street_2",
                                  "Address Type":"Address_Type",
                                  "Facility Type":"Facility_Type",
                                  "Due Date":"Due_Date",
                                  "Resolution Description":"Resolution_Description",
                                  "Resolution Action Updated Date":"Resolution_Action_Updated_Date",
                                  "Community Board":"Community_Board",
                                  "Park Borough":"Park_Borough"})

Loading reference data for zip codes.

In [ ]:
zip_data = pd.read_csv("../data/nyc-zip-codes.csv")

In [ ]:
zip_data.head()

,Borough,Neighborhood,ZipCode
0,Bronx,Central Bronx,10453
1,Bronx,Central Bronx,10457
2,Bronx,Central Bronx,10460
3,Bronx,Bronx Park and Fordham,10458
4,Bronx,Bronx Park and Fordham,10467


In [ ]:
unique_zips = zip_data.ZipCode.unique()

In [ ]:
def check_zip(z):
    
  if z in unique_zips:
        return str(int(z)) 
  
  else:
        return 'Unspecified'

In [ ]:
pd_data.Incident_Zip = pd_data.Incident_Zip.map(lambda x: check_zip(x))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
len(pd_data.Incident_Zip.unique())

179

In [ ]:
pd_data.Incident_Zip.unique()

array(['11421', '10468', '11214', '11229', '11420', '10469', '10456',
       '11217', 'Unspecified', '11357', '11369', '11215', '11210',
       '11233', '10034', '11103', '11220', '11368', '11361', '10016',
       '10463', '11366', '10032', '11419', '10457', '11207', '11226',
       '11219', '11379', '10025', '11435', '11212', '11236', '11230',
       '10462', '11105', '10031', '11208', '10002', '10019', '11209',
       '10459', '10312', '11205', '10309', '10451', '10021', '11203',
       '11102', '10465', '11355', '10460', '10314', '10464', '11218',
       '11414', '11385', '11204', '11206', '11237', '10029', '10301',
       '10452', '10023', '10022', '11691', '10466', '10473', '11373',
       '10075', '10003', '10011', '11358', '10454', '11378', '10305',
       '11377', '11375', '10012', '11211', '11429', '10040', '11432',
       '11106', '11231', '11238', '11234', '11372', '11356', '10467',
       '11235', '11223', '11213', '11370', '10001', '10065', '10014',
       '10461', '11413'

Separating the rows with Unspecified zip values and valid zip values.

In [ ]:
unspecified_zip_data = pd_data.loc[pd_data['Incident_Zip'] == 'Unspecified']

In [ ]:
unspecified_zip_data.head()

,Unnamed: 0,Unique Key,Created_Date,Closed_Date,Agency,Agency_Name,Complaint_Type,Descriptor,Location_Type,Incident_Zip,Incident_Address,Street_Name,Cross_Street_1,Cross_Street_2,Intersection_Street_1,Intersection_Street_2,Address_Type,City,Landmark,Facility_Type,Status,Due_Date,Resolution_Description,Resolution_Action_Updated_Date,Community_Board,BBL,Borough,X Coordinate (State Plane),Y Coordinate (State Plane),Open Data Channel Type,Park Facility Name,Park_Borough,Vehicle Type,Taxi Company Borough,Taxi Pick Up Location,Bridge Highway Name,Bridge Highway Direction,Road Ramp,Bridge Highway Segment,Latitude,Longitude,Location
9,9,51561680,08/17/2021 05:08:00 PM,08/18/2021 09:46:34 AM,DOT,Department of Transportation,Broken Parking Meter,Coin or Card Did Not Register,Street,Unspecified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Closed,NaN,The Department of Transportation inspected the...,08/18/2021 09:46:38 AM,0 Unspecified,NaN,Unspecified,NaN,NaN,ONLINE,Unspecified,Unspecified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14,14,51562435,08/17/2021 07:40:27 PM,08/17/2021 11:59:03 PM,NYPD,New York City Police Department,Encampment,NaN,Highway,Unspecified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Closed,NaN,The Police Department visited the location and...,08/17/2021 11:59:06 PM,Unspecified QUEENS,NaN,QUEENS,NaN,NaN,PHONE,Unspecified,QUEENS,NaN,NaN,NaN,Belt Pkwy,West/Brooklyn Bound,Roadway,Lefferts Blvd Aqueduct Racetrack (Exit 18B) - ...,NaN,NaN,NaN
29,29,51563214,08/18/2021 12:48:20 AM,08/18/2021 02:51:54 AM,NYPD,New York City Police Department,Encampment,NaN,Subway,Unspecified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Closed,NaN,The Police Department visited the location and...,08/18/2021 02:51:58 AM,Unspecified QUEENS,NaN,QUEENS,1012386.0,212417.0,MOBILE,Unspecified,QUEENS,NaN,NaN,NaN,R,M R to Forest Hills - Jamaica,NaN,Platform,40.749667,-73.898453,"(40.74966739007307, -73.89845319094752)"
34,34,51563637,08/18/2021 11:35:00 AM,08/18/2021 10:15:00 PM,DOT,Department of Transportation,Street Light Condition,Street Light Out,NaN,Unspecified,NaN,NaN,NaN,NaN,COLLEGE AVE,164 ST E,INTERSECTION,NaN,NaN,NaN,Closed,NaN,Service Request status for this request is ava...,08/18/2021 10:15:00 PM,Unspecified BRONX,NaN,BRONX,NaN,NaN,UNKNOWN,Unspecified,BRONX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40,40,50472100,05/04/2021 08:47:12 PM,05/14/2021 01:58:18 PM,DOT,Department of Transportation,Highway Condition,Pothole - Highway,Highway,Unspecified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Closed,NaN,The Department of Transportation has completed...,05/14/2021 01:58:21 PM,0 Unspecified,NaN,Unspecified,NaN,NaN,ONLINE,Unspecified,Unspecified,NaN,NaN,NaN,BQE/Gowanus Expwy,East/Bronx Bound,Roadway,Eastern Long Is Riverhead (Exit 35E) - Grand C...,NaN,NaN,NaN


In [ ]:
specified_zip_data = pd_data.loc[pd_data['Incident_Zip'] != 'Unspecified']

In [ ]:
specified_zip_data.head()

,Unnamed: 0,Unique Key,Created_Date,Closed_Date,Agency,Agency_Name,Complaint_Type,Descriptor,Location_Type,Incident_Zip,Incident_Address,Street_Name,Cross_Street_1,Cross_Street_2,Intersection_Street_1,Intersection_Street_2,Address_Type,City,Landmark,Facility_Type,Status,Due_Date,Resolution_Description,Resolution_Action_Updated_Date,Community_Board,BBL,Borough,X Coordinate (State Plane),Y Coordinate (State Plane),Open Data Channel Type,Park Facility Name,Park_Borough,Vehicle Type,Taxi Company Borough,Taxi Pick Up Location,Bridge Highway Name,Bridge Highway Direction,Road Ramp,Bridge Highway Segment,Latitude,Longitude,Location
0,0,51560911,08/17/2021 07:34:45 AM,08/17/2021 09:34:29 AM,NYPD,New York City Police Department,Illegal Parking,Blocked Sidewalk,Street/Sidewalk,11421,92-13 92 AVENUE,92 AVENUE,92 STREET,WOODHAVEN BOULEVARD,92 STREET,WOODHAVEN BOULEVARD,NaN,WOODHAVEN,92 AVENUE,NaN,Closed,NaN,The Police Department issued a summons in resp...,08/17/2021 09:34:31 AM,09 QUEENS,4.089860e+09,QUEENS,1025640.0,190277.0,MOBILE,Unspecified,QUEENS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.688846,-73.850754,"(40.68884640295723, -73.85075384808735)"
1,1,51560923,08/17/2021 06:04:48 PM,08/17/2021 07:25:36 PM,NYPD,New York City Police Department,Illegal Parking,Blocked Sidewalk,Street/Sidewalk,10468,CLAFLIN AVENUE,CLAFLIN AVENUE,CLAFLIN AVENUE,WEST 195 STREET,CLAFLIN AVENUE,WEST 195 STREET,NaN,NaN,NaN,NaN,Closed,NaN,The Police Department responded and upon arriv...,08/17/2021 07:25:44 PM,08 BRONX,NaN,BRONX,1011740.0,256343.0,PHONE,Unspecified,BRONX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.870234,-73.900605,"(40.87023400587434, -73.90060480716431)"
2,2,50469886,05/04/2021 11:40:00 AM,07/21/2021 01:55:00 PM,DOT,Department of Transportation,Street Light Condition,Street Light Out,NaN,11214,NaN,NaN,NaN,NaN,NEW UTRECHT AVENUE,84 STREET,INTERSECTION,BROOKLYN,NaN,NaN,Closed,NaN,Service Request status for this request is ava...,07/21/2021 01:55:00 PM,11 BROOKLYN,NaN,BROOKLYN,983735.0,161202.0,UNKNOWN,Unspecified,BROOKLYN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.609139,-74.001855,"(40.60913860285155, -74.00185479490075)"
3,3,50470191,05/04/2021 04:31:32 PM,NaN,DPR,Department of Parks and Recreation,New Tree Request,For One Address,Street,11229,2041 EAST 18 STREET,EAST 18 STREET,AVENUE T,AVENUE U,AVENUE T,AVENUE U,NaN,BROOKLYN,EAST 18 STREET,NaN,In Progress,NaN,NaN,NaN,15 BROOKLYN,3.073230e+09,BROOKLYN,997085.0,158030.0,UNKNOWN,Unspecified,BROOKLYN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.600423,-73.953780,"(40.600422830424975, -73.95378022131071)"
4,4,50470218,05/05/2021 01:47:05 AM,10/01/2021 10:53:17 AM,DOHMH,Department of Health and Mental Hygiene,Rodent,Signs of Rodents,1-2 Family Dwelling,11420,115-44 135 STREET,135 STREET,115 AVENUE,116 AVENUE,115 AVENUE,116 AVENUE,ADDRESS,South Ozone Park,135 STREET,NaN,Closed,NaN,This service request was closed because the De...,05/05/2021 01:47:05 AM,10 QUEENS,4.116770e+09,QUEENS,1038085.0,187070.0,MOBILE,Unspecified,QUEENS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.679977,-73.805905,"(40.67997699346374, -73.8059047916443)"


In [ ]:
unspecified_zip_data = unspecified_zip_data.reset_index()

In [ ]:
specified_zip_data = specified_zip_data.reset_index()

In [ ]:
zip_data['ZipCode'] = zip_data['ZipCode'].map(str)

In [ ]:
zip_data.loc[zip_data['ZipCode'] == '11209', 'Borough'].iloc[0]

'Brooklyn'

In [ ]:
specified_zip_data.Borough[4]

'QUEENS'

In [ ]:
zip_data['Borough'] = zip_data['Borough'].str.upper()

CHecking unique values of Boroughs in reference data.

In [ ]:
zip_data.Borough.unique()

array(['BRONX', 'BROOKLYN', 'MANHATTAN', 'QUEENS', 'STATEN ISLAND'],
      dtype=object)

Mapping the Boroughs to zip codes using reference data. If mapping found to be wrong, replace with valid value of Borough from reference data.

In [ ]:
for i in range(len(specified_zip_data)):
  if (specified_zip_data.Borough[i] == zip_data.loc[zip_data['ZipCode'] == specified_zip_data.Incident_Zip[i], 'Borough'].iloc[0]):
    continue
  else:
    specified_zip_data.Borough[i] = zip_data.loc[zip_data['ZipCode'] == specified_zip_data.Incident_Zip[i], 'Borough'].iloc[0]

Unique values of Boroughs in the dataset after Mapping

In [ ]:
specified_zip_data.Borough.unique()

array(['MANHATTAN', 'BROOKLYN', 'BRONX', 'STATEN ISLAND', 'QUEENS'],
      dtype=object)